In [1]:
import copy
from tqdm import tqdm

import platform, multiprocessing
from multiprocessing import Pool

import numpy as np
from astropy.io import fits
import Simpler_Classifier

from importlib import reload 


[INFO]: No release version set. Setting default to DR17
[WARNING]: path /Users/runquanguan/sas/mangawork/manga/spectro/redux/v3_1_1/drpall-v3_1_1.fits cannot be found. Setting drpall to None. (MarvinUserWarning)
[WARNING]: path /Users/runquanguan/sas/mangawork/manga/spectro/analysis/v3_1_1/3.1.0/dapall-v3_1_1-3.1.0.fits cannot be found. Setting dapall to None. (MarvinUserWarning)
[WARNING]: Passing field metadata as keyword arguments is deprecated. Use the explicit `metadata=...` argument instead. Additional metadata: {'location': 'view_args'} (RemovedInMarshmallow4Warning)
[WARNING]: The 'missing' argument to fields is deprecated. Use 'load_default' instead. (RemovedInMarshmallow4Warning)



my_module = reload(Simpler_Classifier) # reload of Simpler_Classifier without restart

In [2]:
#HEADER_PATH = '/Users/runquanguan/Desktop/0823samples.fits'
HEADER_PATH = '/Users/runquanguan/Documents/bicone_training_set.fits'
#HEADER_PATH = '/Users/runquanguan/Documents/dapall_excluded.fits'
#HEADER_PATH = '/Users/runquanguan/Desktop/0824MPL11.fits'
hdul = fits.open(HEADER_PATH)
hdu = hdul[1].data
plateifu = hdu['PLATEIFU']


results = []
for data in plateifu:
    result = Simpler_Classifier.fit_test(data)
    results.append(result)

In [3]:
if __name__ == '__main__':
    if platform.system() == "Darwin":
        multiprocessing.set_start_method('spawn')
        pool = multiprocessing.Pool(processes=10)
        results = []
        for result in tqdm(pool.imap_unordered(Simpler_Classifier.fit_test, plateifu), total = len(plateifu)):
            results.append(result)

        pool.close()
        pool.join() 


100%|██████████| 156/156 [02:25<00:00,  1.07it/s]


In [4]:
result_copy = copy.deepcopy(results)

In [5]:
clean = []
for i in result_copy:
    if i != None:
        clean.append(i[0])
len(clean)

154

In [6]:
store = []
for i in result_copy:
    if i != None:
        store.append(i)

In [7]:
miss_list = []
for data in list(plateifu):
    if data in clean:
        pass
    else:
        miss_list.append(data)

len(miss_list)

2

In [8]:
miss_results = []
for i in tqdm(miss_list):
    result = Simpler_Classifier.fit_test(i)
    miss_results.append(result)

100%|██████████| 2/2 [00:11<00:00,  5.54s/it]


In [9]:
clean_2 = []
for i in miss_results:
    if i != None:
        clean_2.append(i[0])
len(clean_2)

2

In [10]:
for i in miss_results:
    if i != None:
        store.append(i)
len(store)

156

In [11]:
miss_list_2 = []
for i in miss_list:
    if i in clean_2:
        pass
    else:
        miss_list_2.append(i)
len(miss_list_2)

0

In [12]:
miss_results_2 = []
for i in tqdm(miss_list_2):
    result = Simpler_Classifier.fit_test(i)
    miss_results_2.append(result)

0it [00:00, ?it/s]


In [13]:
for i in miss_results_2:
    store.append(i)

In [14]:
len(store)

156

In [15]:
pla_re = []
fit = []
center = []
scale = []
sig = []

for i in store:
    if i != None:
        pla_re.append(i[0])
        fit.append(i[1])
        center.append(i[2])
        scale.append(i[3])
        sig.append(i[4])

        
    else:
        pass

        
#img_array = np.array(value)
#ima_res = img_array.imag

In [16]:
if len(store) == 156:

    c1 = fits.Column(name='plateifu', array=np.array(pla_re), format='20A')
    c2 = fits.Column(name='scale_min', array=np.array(scale), format='E')
    c3 = fits.Column(name='center_max', array=np.array(center), format='E')
    c4 = fits.Column(name='sig_max', array=np.array(sig), format='E')
    c5 = fits.Column(name='fitness', array=np.array(fit), format='E')

    t = fits.BinTableHDU.from_columns([c1, c2, c3, c4, c5])
    t.writeto('0831sample_sqrt_fit.fits')
else:
    pass

In [17]:
len(store)

156

for i in store:
    x = [1,2,3,4]
    y = []
    for n in i[1][1]:
        nn = np.sign(n) * np.log10(abs(n))
        y.append(nn)
    plt.plot(x, y)
    
plt.show()

y = []
for i in store:
    a=i[1][1][0]/i[1][1][2]
    b=i[1][1][1]/i[1][1][3]
    y.append([i[0], abs(a-b)])
    
    
#x = np.linspace(0,156,156)
#y.sort()
y.sort()